# 3 level classification architecture
Source: https://www.kaggle.com/svpons/airbnb-recruiting-new-user-bookings/three-level-classification-architecture/comments


In [1]:
import numpy as  np
from sklearn.metrics import log_loss
from sklearn.base import BaseEstimator
from scipy.optimize import minimize

## First ensemble technique (EN_optA)
Given a set of predictions $X_1,X_2,...,X_n$, it computes the optimal set of weights $w_1,w_2,...,w_n$; such that minimizes $log\_loss(y_T,y_E)$, where $y_E=\sum_{i=1}^n X_i\times w_i$ and $y_T$ is the true solution.

In [2]:
def objf_ens_optA(w, Xs, y, n_class=5):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (5 in Animal Shelter competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class=5):
        super(EN_optA, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        Xs = np.hsplit(X, X.shape[1] / self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA, x0, args=(Xs, y, self.n_class), 
                       method='SLSQP', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred  

## Second ensemble technique (EN_optB)
Given a set of predictions $X_1,X_2$, where each $X_i$ has $m=5$ classes, i.e. $X_i=X_{i1},...,X_{im}$. The algorithm finds the optimal set of weights $w_{11},...,w_{nm}$ such that minimizes $log\_loss(y_T,y_E)$ where $y_E=\sum_{i=1}^n\sum_{j=1}^m X_{ij}\times w_{ij}$ and $y_T$ is the true solution.

In [3]:
def objf_ens_optB(w, Xs, y, n_class=5):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 5
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class=5):
        super(EN_optB, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB, x0, args=(Xs, y, self.n_class), 
                       method='L-BFGS-B', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred      

## Use them

In [4]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from xgboost.sklearn import XGBClassifier

import pandas as pd

#fixing random state
random_state = 123

In [5]:
# data
train = pd.read_csv("data/train_clean.csv")
test = pd.read_csv("data/test_clean.csv")

In [6]:
# split data
train_X = train.ix[:, train.columns != "OutcomeType"]
train_Y = train["OutcomeType"]

mapping = {
    "Adoption": 0,
    "Died": 1,
    "Euthanasia": 2,
    "Return_to_owner": 3,
    "Transfer": 4
}
train_Y = train_Y.replace(mapping)

test_ID = test["ID"]
test_X = test.ix[:, test.columns != "ID"]

# if using the smote data set
test_X.columns = [x.replace(" ",".") for x in test_X.columns]

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)

train_X = train_X.astype(float)
test_X = test_X.astype(float)

(26728, 452)
(26728,)
(11456, 452)


In [7]:
# spliting data into train and test sets.
X, X_test, y, y_test = train_test_split(train_X, train_Y, test_size=0.2, 
                                        random_state=random_state)

# spliting train data into training and validation sets.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, 
                                                      random_state=random_state)

print('Data shape:')
print('X_train: %s, X_valid: %s, X_test: %s \n' %(X_train.shape, X_valid.shape, 
                                                  X_test.shape))

Data shape:
X_train: (16036, 452), X_valid: (5346, 452), X_test: (5346, 452) 



## First layer (indivisual classifiers)
All classifiers are applied twice:
- train on (X_train,y_train) and predicting on (X_valid)
- train on (X,y) and predicting on (X_test)

In [18]:
# define the classifiers
clfs = {# 'LR'  : LogisticRegression(random_state=random_state), 
        # 'SVM' : SVC(probability=True, random_state=random_state), 
        'GBM1' : GradientBoostingClassifier(n_estimators=100, 
                                            learning_rate=0.1,
                                            random_state=random_state), 
        'GBM2' : GradientBoostingClassifier(n_estimators=150, 
                                            learning_rate=0.1,
                                            random_state=random_state), 
        'GBM3' : GradientBoostingClassifier(n_estimators=200, 
                                            learning_rate=0.1,
                                            random_state=random_state), 
        'GBM4' : GradientBoostingClassifier(n_estimators=100, 
                                            max_depth = 6,
                                            learning_rate=0.2,
                                            random_state=random_state), 
        'GBM5' : GradientBoostingClassifier(n_estimators=200, 
                                            max_depth = 6,
                                            learning_rate=0.2,
                                            random_state=random_state), 
        'RF'  : RandomForestClassifier(n_estimators=1000, n_jobs=-1, 
                                       random_state=random_state), 
        'ETC' : ExtraTreesClassifier(n_estimators=1000, n_jobs=-1, 
                                     random_state=random_state),
        # 'KNN' : KNeighborsClassifier(n_neighbors=30)
       }
    
# predictions on the validation and test sets
p_valid = []
p_test = []
   
print('Performance of individual classifiers (1st layer) on X_test')   
print('------------------------------------------------------------')
   
for nm, clf in clfs.items():
    # first run. training on (X_train, y_train) and predicting on X_valid.
    clf.fit(X_train, y_train)
    yv = clf.predict_proba(X_valid)
    p_valid.append(yv)
        
    # second run. training on (X, y) and predicting on X_test.
    clf.fit(X, y)
    yt = clf.predict_proba(X_test)
    p_test.append(yt)
       
    #Printing out the performance of the classifier
    print('{:10s} {:2s} {:1.7f}'.format('%s: ' %(nm), 'logloss  =>', log_loss(y_test, yt)))
print('')

Performance of individual classifiers (1st layer) on X_test
------------------------------------------------------------
GBM3:      logloss  => 0.7777456
GBM1:      logloss  => 0.7828621
GBM4:      logloss  => 0.7845642
GBM2:      logloss  => 0.7789858
RF:        logloss  => 0.8173097
ETC:       logloss  => 0.8671636
GBM5:      logloss  => 0.8118395



## Second layer (optimization based ensembles)
Predictions on X_valid are used as training set (XV) and predictions on X_test are used as test set (XT). EN_optA, EN_optB and their calibrated versions are applied.

In [19]:
print('Performance of optimization based ensemblers (2nd layer) on X_test')   
print('------------------------------------------------------------')
    
#Creating the data for the 2nd layer.
XV = np.hstack(p_valid)
XT = np.hstack(p_test)  
        
#EN_optA
enA = EN_optA(5)
enA.fit(XV, y_valid)
w_enA = enA.w
y_enA = enA.predict_proba(XT)
print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA)))
    
#Calibrated version of EN_optA 
cc_optA = CalibratedClassifierCV(enA, method='isotonic')
cc_optA.fit(XV, y_valid)
y_ccA = cc_optA.predict_proba(XT)
print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA:', 'logloss  =>', log_loss(y_test, y_ccA)))
        
#EN_optB
enB = EN_optB(5) 
enB.fit(XV, y_valid)
w_enB = enB.w
y_enB = enB.predict_proba(XT)
print('{:20s} {:2s} {:1.7f}'.format('EN_optB:', 'logloss  =>', log_loss(y_test, y_enB)))

#Calibrated version of EN_optB
cc_optB = CalibratedClassifierCV(enB, method='isotonic')
cc_optB.fit(XV, y_valid)
y_ccB = cc_optB.predict_proba(XT)  
print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB:', 'logloss  =>', log_loss(y_test, y_ccB)))
print('')

Performance of optimization based ensemblers (2nd layer) on X_test
------------------------------------------------------------
EN_optA:             logloss  => 0.7552470
Calibrated_EN_optA:  logloss  => 0.7773318
EN_optB:             logloss  => 0.7514325
Calibrated_EN_optB:  logloss  => 0.7705297



## Third layer (weighted average)
Simple weighted average of the previous 4 predictions.

In [20]:
y_3l = (y_enA * 4./9.) + (y_ccA * 2./9.) + (y_enB * 2./9.) + (y_ccB * 1./9.)
print('{:20s} {:2s} {:1.7f}'.format('3rd_layer:', 'logloss  =>', log_loss(y_test, y_3l)))

3rd_layer:           logloss  => 0.7508924


## Plotting the weights of each ensemble
In the case of EN_optA, there is a weight for each prediction and in the case of EN_optB there is a weight for each class for each prediction.

In [22]:
from tabulate import tabulate
print('               Weights of EN_optA:')
print('|---------------------------------------------|')
wA = np.round(w_enA, decimals=2).reshape(1,-1)
print(tabulate(wA, headers=clfs.keys(), tablefmt="orgtbl"))
print('')
print('                                    Weights of EN_optB:')
print('|-------------------------------------------------------------------------------|')
wB = np.round(w_enB.reshape((-1,5)), decimals=2)
wB = np.hstack((np.array(list(clfs.keys()), dtype=str).reshape(-1,1), wB))
print(tabulate(wB, headers=['y%s'%(i) for i in range(5)], tablefmt="orgtbl"))

               Weights of EN_optA:
|---------------------------------------------|
|   GBM3 |   GBM1 |   GBM4 |   GBM2 |   RF |   ETC |   GBM5 |
|--------+--------+--------+--------+------+-------+--------|
|   0.42 |      0 |      0 |    0.1 | 0.18 |     0 |    0.3 |

                                    Weights of EN_optB:
|-------------------------------------------------------------------------------|
|      |   y0 |   y1 |   y2 |   y3 |   y4 |
|------+------+------+------+------+------|
| GBM3 | 0.52 | 0    | 0.62 | 0.52 | 0.12 |
| GBM1 | 0    | 0.84 | 0    | 0    | 0    |
| GBM4 | 0.22 | 0    | 0    | 0    | 0    |
| GBM2 | 0    | 0    | 0.12 | 0    | 0.05 |
| RF   | 0.09 | 0.16 | 0.09 | 0.3  | 0.22 |
| ETC  | 0    | 0    | 0    | 0    | 0    |
| GBM5 | 0.17 | 0    | 0.17 | 0.18 | 0.61 |


## train the architecture on all train data

In [ ]:
# 1st layer

# predictions on the validation and test sets
p_valid = []
p_test = []
   
print('Performance of individual classifiers (1st layer) on X_test')   
print('------------------------------------------------------------')
   
for nm, clf in clfs.items():
    # first run. training on (X_train, y_train) and predicting on X_valid.
    clf.fit(train_X, train_Y)
    yv = clf.predict_proba(test_X)
    p_valid.append(yv)
        
    # second run. training on (X, y) and predicting on X_test.
    clf.fit(train_X, train_Y)
    yt = clf.predict_proba(test_X)
    p_test.append(yt)
       
    #Printing out the performance of the classifier
    # print('{:10s} {:2s} {:1.7f}'.format('%s: ' %(nm), 'logloss  =>', log_loss(y_test, yt)))
print('')

Performance of individual classifiers (1st layer) on X_test
------------------------------------------------------------
